# Object detection using yolov5 and common objects datasets from roboflow
<p> by Abdulhameed Yunusa </p> - <a href="https://github.com/ayhameed">Github</a>

## Connect to gdrive

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
# Verify connection
!ls "/content/drive/My Drive"

# Setting Roboflow API Key

## Store API key

In [9]:
# Store API key securely
import os
os.environ['ROBOFLOW_API_KEY'] = 'place-your-api-key-here'

In [10]:
# Get API key
api_key = os.environ.get('ROBOFLOW_API_KEY')

## Initialize Roboflow

In [ ]:
# Install roboflow if not already installed
!pip install roboflow

In [7]:
# Import Roboflow
from roboflow import Roboflow

In [11]:
# Create an Instance of the Roboflow
rf = Roboflow(api_key=api_key)

In [ ]:
# Test the connection
workspace = rf.workspace()
print("Successfully connected to Roboflow!")

## Seting up the Enviroment

In [ ]:
# Install required packages
!pip install torch torchvision
!pip install opencv-python-headless

In [65]:
%matplotlib inline

In [ ]:
!git clone https://github.com/ultralytics/yolov5

In [ ]:
# Install YOLOv5 requirements
%cd yolov5
!pip install -r requirements.txt

In [51]:
# Import all necessary libraries
import torch
import cv2
from IPython.display import display, Image
from google.colab import files
import matplotlib.pyplot as plt
import matplotlib.patches as patches
from matplotlib.patches import Rectangle
import matplotlib.image as mpimg
from pathlib import Path
import numpy as np
import os

## Load Pre-trained Model

In [ ]:
# Load YOLOv5s (small model) with pre-trained weights
model = torch.hub.load('ultralytics/yolov5', 'yolov5s', pretrained=True)

## Object detection function

In [69]:
def detect_and_display_objects(image_path):
    """
    Detect objects in an image and display results
    """
    # Load model
    model = torch.hub.load('ultralytics/yolov5', 'yolov5s', pretrained=True)

    # Perform inference
    results = model(image_path)

    # Get prediction results
    result_image = results.render()[0]  # returns RGB numpy array

    # Convert from RGB to BGR (for cv2)
    result_image = cv2.cvtColor(result_image, cv2.COLOR_RGB2BGR)

    # Display using matplotlib
    plt.figure(figsize=(15,10))
    plt.imshow(cv2.cvtColor(result_image, cv2.COLOR_BGR2RGB))
    plt.axis('off')
    plt.show()

    # Print detected objects with confidence
    df = results.pandas().xyxy[0]
    if len(df) > 0:
        print("\nDetected objects:")
        for idx, row in df.iterrows():
            print(f"{row['name']} (Confidence: {row['confidence']:.2%})")
    else:
        print("\nNo objects detected")

    return results

In [ ]:
# For new image upload
print("Please upload an image...")
uploaded = files.upload()
filename = list(uploaded.keys())[0]

# Detect and display
results = detect_and_display_objects(filename)


In [ ]:
# Optional: Save the results
save_path = 'detection_result_2.jpg'
cv2.imwrite(save_path, cv2.cvtColor(results.render()[0], cv2.COLOR_RGB2BGR))
print(f"\nResults saved as '{save_path}'")

In [ ]:
def display_with_confidence(image_path, confidence_threshold=0.25):

    # Load model
    model = torch.hub.load('ultralytics/yolov5', 'yolov5s', pretrained=True)

    # Set confidence threshold
    model.conf = confidence_threshold

    # Perform inference
    results = model(image_path)

    # Get results as pandas dataframe
    df = results.pandas().xyxy[0]

    # Load the image
    img = Image.open(image_path)

    # Plot the image with bounding boxes
    fig, ax = plt.subplots(1)
    ax.imshow(img)

    # Add bounding boxes
    for idx, row in df.iterrows():
        xmin, ymin, xmax, ymax = row['xmin'], row['ymin'], row['xmax'], row['ymax']
        rect = patches.Rectangle((xmin, ymin), xmax - xmin, ymax - ymin, linewidth=2, edgecolor='r', facecolor='none')
        ax.add_patch(rect)
        ax.text(xmin, ymin, f"{row['name']} {row['confidence']:.2%}", color='white', fontsize=12,
                bbox=dict(facecolor='red', alpha=0.5))

    # Display the result
    plt.show()

    # Print detailed results
    print("\nDetailed Detection Results:")
    for idx, row in df.iterrows():
        print(f"Object {idx+1}:")
        print(f"- Class: {row['name']}")
        print(f"- Confidence: {row['confidence']:.2%}")
        print(f"- Bounding Box: ({row['xmin']:.0f}, {row['ymin']:.0f}) to ({row['xmax']:.0f}, {row['ymax']:.0f})")
        print()

# Example usage
display_with_confidence('detection_result_2.jpg', confidence_threshold=0.25)